# Topological Feature Comparison Pipeline

This notebook compares aggregated topological features between medOn and medOff states for Parkinson's disease patients.

## Analysis Goals
- Compare medOn vs medOff states using topological features
- Examine differences between dominant and non-dominant hemispheres
- Identify which features (H0, H1, H2, H3) are most discriminative
- Analyze hold vs resting state differences

# Part 1: Multi-Patient Analysis

Load and compare features across all available patients.

## 1. Import Libraries

In [54]:
# Standard libraries
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import os

# Statistical analysis
from scipy import stats
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 4)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Define Patient IDs and Data Paths

In [55]:
# Base directory
BASE_DIR = Path('.')

# Patient IDs organized by hold type
# holdL: Subject raised LEFT arm (RIGHT hemisphere is dominant)
# holdR: Subject raised RIGHT arm (LEFT hemisphere is dominant)
PATIENTS_HOLD_L = ['0cGdk9', '2IU8mi', 'AB2PeX', 'AbzsOg', 'FYbcap', 
                   'PuPVlx', 'QZTsn6', 'dCsWjQ', 'gNX5yb', 'i4oK0F']

PATIENTS_HOLD_R = ['2IhVOz', 'BYJoWR', 'VopvKx', 'jyC0j3']

# All patients
ALL_PATIENTS = PATIENTS_HOLD_L + PATIENTS_HOLD_R

print(f"Total patients: {len(ALL_PATIENTS)}")
print(f"  - holdL (left arm raised): {len(PATIENTS_HOLD_L)}")
print(f"  - holdR (right arm raised): {len(PATIENTS_HOLD_R)}")

Total patients: 14
  - holdL (left arm raised): 10
  - holdR (right arm raised): 4


## 3. Load Aggregated Features - Single Patient Example

First, let's load one patient's data to understand the structure.

In [56]:
# Load aggregated features for one patient (example: i4oK0F)
patient_id = 'i4oK0F'
patient_dir = BASE_DIR / patient_id

# Find aggregated features file
agg_files = list(patient_dir.glob('aggregated_features*.pkl'))
if agg_files:
    agg_file = agg_files[0]
    print(f"Loading: {agg_file}")
    
    with open(agg_file, 'rb') as f:
        patient_data = pickle.load(f)
    
    # Check structure
    print(f"\nTop-level keys: {list(patient_data.keys())}")
    
    if '_metadata' in patient_data:
        print(f"\nMetadata: {patient_data['_metadata']}")
    
    if 'medOn' in patient_data:
        print(f"\nmedOn keys: {list(patient_data['medOn'].keys())}")
        
        # Show example feature keys
        if 'left_hold' in patient_data['medOn']:
            print(f"\nleft_hold feature keys (sample):")
            feature_keys = list(patient_data['medOn']['left_hold'].keys())
            print(feature_keys[:10])  # Show first 10
    
    print("\n✓ Data loaded successfully")
else:
    print(f"No aggregated features found for {patient_id}")

Loading: i4oK0F/aggregated_features_holdL.pkl

Top-level keys: ['medOn', 'medOff', '_metadata']

Metadata: {'hold_type': 'holdL', 'hold_suffix': '_holdL'}

medOn keys: ['left_hold', 'left_resting', 'right_hold', 'right_resting', 'dominant_hold', 'nondominant_hold', 'dominant_resting', 'nondominant_resting']

left_hold feature keys (sample):
['hemisphere', 'condition', 'persistence_entropy_mean', 'persistence_entropy_std', 'h0_feature_count_mean', 'h0_feature_count_std', 'h0_avg_lifespan_mean', 'h0_avg_lifespan_std', 'h0_max_lifespan_mean', 'h0_max_lifespan_std']

✓ Data loaded successfully


## 4. Extract Features - medOn State

In [57]:
# Extract medOn features using recommended dominant/nondominant mapping
if 'medOn' in patient_data:
    # Dominant hemisphere (active during hold task)
    medOn_dominant_hold = patient_data['medOn'].get('dominant_hold', {})
    medOn_dominant_resting = patient_data['medOn'].get('dominant_resting', {})
    
    # Non-dominant hemisphere
    medOn_nondominant_hold = patient_data['medOn'].get('nondominant_hold', {})
    medOn_nondominant_resting = patient_data['medOn'].get('nondominant_resting', {})
    
    # Original left/right naming (for reference)
    medOn_left_hold = patient_data['medOn'].get('left_hold', {})
    medOn_left_resting = patient_data['medOn'].get('left_resting', {})
    medOn_right_hold = patient_data['medOn'].get('right_hold', {})
    medOn_right_resting = patient_data['medOn'].get('right_resting', {})
    
    print("✓ medOn features extracted")
    print(f"  Dominant hold features: {len(medOn_dominant_hold)}")
    print(f"  Non-dominant hold features: {len(medOn_nondominant_hold)}")

✓ medOn features extracted
  Dominant hold features: 59
  Non-dominant hold features: 59


## 5. Extract Features - medOff State

In [58]:
# Extract medOff features using recommended dominant/nondominant mapping
if 'medOff' in patient_data:
    # Dominant hemisphere (active during hold task)
    medOff_dominant_hold = patient_data['medOff'].get('dominant_hold', {})
    medOff_dominant_resting = patient_data['medOff'].get('dominant_resting', {})
    
    # Non-dominant hemisphere
    medOff_nondominant_hold = patient_data['medOff'].get('nondominant_hold', {})
    medOff_nondominant_resting = patient_data['medOff'].get('nondominant_resting', {})
    
    # Original left/right naming (for reference)
    medOff_left_hold = patient_data['medOff'].get('left_hold', {})
    medOff_left_resting = patient_data['medOff'].get('left_resting', {})
    medOff_right_hold = patient_data['medOff'].get('right_hold', {})
    medOff_right_resting = patient_data['medOff'].get('right_resting', {})
    
    print("✓ medOff features extracted")
    print(f"  Dominant hold features: {len(medOff_dominant_hold)}")
    print(f"  Non-dominant hold features: {len(medOff_nondominant_hold)}")

✓ medOff features extracted
  Dominant hold features: 59
  Non-dominant hold features: 59


## 6. Helper Functions for Safe Formatting

In [ ]:
def safe_format(value, format_spec='.4f', default='N/A'):
    """
    Safely format a value, returning default if value is None or not numeric.
    
    Args:
        value: Value to format (can be scalar or array)
        format_spec: Format specification (e.g., '.4f', '.2f')
        default: Default value if formatting fails
    
    Returns:
        str: Formatted string
    """
    # Check if value is None
    if value is None:
        return default
    
    # Check if value is a numpy array or list
    if isinstance(value, (np.ndarray, list)):
        return default
    
    # Check if value is 'N/A' string
    if isinstance(value, str) and value == 'N/A':
        return default
    
    try:
        # Use format() with the spec string
        return format(value, format_spec)
    except (ValueError, TypeError):
        return default

def safe_diff(val1, val2, format_spec='.4f', default='N/A'):
    """
    Safely compute and format difference between two values.
    
    Args:
        val1: First value
        val2: Second value
        format_spec: Format specification
        default: Default value if computation fails
    
    Returns:
        str: Formatted difference
    """
    # Check if either value is None
    if val1 is None or val2 is None:
        return default
    
    # Check if either value is an array or list
    if isinstance(val1, (np.ndarray, list)) or isinstance(val2, (np.ndarray, list)):
        return default
    
    # Check if either value is 'N/A' string
    if (isinstance(val1, str) and val1 == 'N/A') or (isinstance(val2, str) and val2 == 'N/A'):
        return default
    
    try:
        diff = val1 - val2
        return format(diff, format_spec)
    except (ValueError, TypeError):
        return default

print("✓ Helper functions defined")

## 7. Compare Key Features - Single Patient

In [61]:
# Compare persistence entropy between medOn and medOff
if medOn_dominant_hold and medOff_dominant_hold:
    print(f"Patient: {patient_id}")
    print(f"Hold type: {patient_data.get('_metadata', {}).get('hold_type', 'unknown')}\n")
    
    # Persistence entropy comparison
    print("Persistence Entropy (Mean):")
    medOn_pe = medOn_dominant_hold.get('persistence_entropy_mean')
    medOff_pe = medOff_dominant_hold.get('persistence_entropy_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_pe, '.4f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_pe, '.4f')}")
    print(f"  Difference: {safe_diff(medOn_pe, medOff_pe, '.4f')}\n")
    
    # Feature count comparison (H1 dimension)
    print("H1 Feature Count (Mean):")
    medOn_h1 = medOn_dominant_hold.get('h1_feature_count_mean')
    medOff_h1 = medOff_dominant_hold.get('h1_feature_count_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h1, '.2f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h1, '.2f')}")
    print(f"  Difference: {safe_diff(medOn_h1, medOff_h1, '.2f')}\n")
    
    # Feature count comparison (H2 dimension)
    print("H2 Feature Count (Mean):")
    medOn_h2 = medOn_dominant_hold.get('h2_feature_count_mean')
    medOff_h2 = medOff_dominant_hold.get('h2_feature_count_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h2, '.2f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h2, '.2f')}")
    print(f"  Difference: {safe_diff(medOn_h2, medOff_h2, '.2f')}\n")
    
    # Lifespan comparison (H1 dimension)
    print("H1 Average Lifespan (Mean):")
    medOn_h1_life = medOn_dominant_hold.get('h1_avg_lifespan_mean')
    medOff_h1_life = medOff_dominant_hold.get('h1_avg_lifespan_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h1_life, '.4f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h1_life, '.4f')}")
    print(f"  Difference: {safe_diff(medOn_h1_life, medOff_h1_life, '.4f')}\n")
    
    # Max lifespan comparison (H1 dimension)
    print("H1 Max Lifespan (Mean):")
    medOn_h1_max = medOn_dominant_hold.get('h1_max_lifespan_mean')
    medOff_h1_max = medOff_dominant_hold.get('h1_max_lifespan_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h1_max, '.4f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h1_max, '.4f')}")
    print(f"  Difference: {safe_diff(medOn_h1_max, medOff_h1_max, '.4f')}\n")
    
    # Birth/Death time comparison (H1 dimension)
    print("H1 Average Birth Time (Mean):")
    medOn_h1_birth = medOn_dominant_hold.get('h1_avg_birth_mean')
    medOff_h1_birth = medOff_dominant_hold.get('h1_avg_birth_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h1_birth, '.4f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h1_birth, '.4f')}")
    print(f"  Difference: {safe_diff(medOn_h1_birth, medOff_h1_birth, '.4f')}\n")
    
    print("H1 Average Death Time (Mean):")
    medOn_h1_death = medOn_dominant_hold.get('h1_avg_death_mean')
    medOff_h1_death = medOff_dominant_hold.get('h1_avg_death_mean')
    print(f"  medOn  dominant hold:  {safe_format(medOn_h1_death, '.4f')}")
    print(f"  medOff dominant hold:  {safe_format(medOff_h1_death, '.4f')}")
    print(f"  Difference: {safe_diff(medOn_h1_death, medOff_h1_death, '.4f')}\n")
    
    # Advanced topological features (arrays - compute L2 norm for comparison)
    print("Advanced Topological Features (Array-based):")
    print("-" * 60)
    
    # Persistence Landscape - compute L2 norm
    medOn_pl = medOn_dominant_hold.get('persistence_landscape_mean')
    medOff_pl = medOff_dominant_hold.get('persistence_landscape_mean')
    if medOn_pl is not None and medOff_pl is not None:
        try:
            medOn_pl_norm = np.linalg.norm(medOn_pl)
            medOff_pl_norm = np.linalg.norm(medOff_pl)
            print(f"Persistence Landscape L2 Norm:")
            print(f"  medOn:  {medOn_pl_norm:.4f}")
            print(f"  medOff: {medOff_pl_norm:.4f}")
            print(f"  Difference: {medOn_pl_norm - medOff_pl_norm:.4f}")
            print(f"  Shape: {medOn_pl.shape}\n")
        except:
            print("Persistence Landscape: (error computing norm)\n")
    else:
        print("Persistence Landscape: (not available)\n")
    
    # Betti Curve - compute L2 norm
    medOn_bc = medOn_dominant_hold.get('betti_curve_mean')
    medOff_bc = medOff_dominant_hold.get('betti_curve_mean')
    if medOn_bc is not None and medOff_bc is not None:
        try:
            medOn_bc_norm = np.linalg.norm(medOn_bc)
            medOff_bc_norm = np.linalg.norm(medOff_bc)
            print(f"Betti Curve L2 Norm:")
            print(f"  medOn:  {medOn_bc_norm:.4f}")
            print(f"  medOff: {medOff_bc_norm:.4f}")
            print(f"  Difference: {medOn_bc_norm - medOff_bc_norm:.4f}")
            print(f"  Shape: {medOn_bc.shape}\n")
        except:
            print("Betti Curve: (error computing norm)\n")
    else:
        print("Betti Curve: (not available)\n")
    
    # Heat Kernel - compute L2 norm
    medOn_hk = medOn_dominant_hold.get('heat_kernel_mean')
    medOff_hk = medOff_dominant_hold.get('heat_kernel_mean')
    if medOn_hk is not None and medOff_hk is not None:
        try:
            medOn_hk_norm = np.linalg.norm(medOn_hk)
            medOff_hk_norm = np.linalg.norm(medOff_hk)
            print(f"Heat Kernel L2 Norm:")
            print(f"  medOn:  {medOn_hk_norm:.4f}")
            print(f"  medOff: {medOff_hk_norm:.4f}")
            print(f"  Difference: {medOn_hk_norm - medOff_hk_norm:.4f}")
            print(f"  Shape: {medOn_hk.shape}")
        except:
            print("Heat Kernel: (error computing norm)")
    else:
        print("Heat Kernel: (not available)")
    
    # Show all available H1 features
    h1_features = [k for k in medOn_dominant_hold.keys() if 'h1_' in k.lower()]
    if h1_features:
        print(f"\n\nAvailable H1 features ({len(h1_features)} total):")
        print(", ".join(sorted(h1_features)))

Patient: i4oK0F
Hold type: holdL

Persistence Entropy (Mean):
  medOn  dominant hold:  6.4704
  medOff dominant hold:  6.4688
  Difference: 0.0016

H1 Feature Count (Mean):
  medOn  dominant hold:  219.40
  medOff dominant hold:  203.40
  Difference: 16.00

H2 Feature Count (Mean):
  medOn  dominant hold:  106.00
  medOff dominant hold:  102.00
  Difference: 4.00

H1 Average Lifespan (Mean):
  medOn  dominant hold:  0.2259
  medOff dominant hold:  0.1816
  Difference: 0.0442

H1 Max Lifespan (Mean):
  medOn  dominant hold:  0.8108
  medOff dominant hold:  0.6565
  Difference: 0.1543

H1 Average Birth Time (Mean):
  medOn  dominant hold:  2.8128
  medOff dominant hold:  2.3163
  Difference: 0.4964

H1 Average Death Time (Mean):
  medOn  dominant hold:  3.0387
  medOff dominant hold:  2.4980
  Difference: 0.5407

Advanced Topological Features (Array-based):
------------------------------------------------------------
Persistence Landscape L2 Norm:
  medOn:  11.8597
  medOff: 10.1940
  Di

## 8. Load All Patients - Batch Loading Function

In [30]:
def load_patient_features(patient_id, base_dir=BASE_DIR):
    """
    Load aggregated features for a patient.
    
    Returns:
        dict: Patient data or None if not found
    """
    patient_dir = base_dir / patient_id
    
    if not patient_dir.exists():
        print(f"⚠ Directory not found: {patient_dir}")
        return None
    
    # Find aggregated features file
    agg_files = list(patient_dir.glob('aggregated_features*.pkl'))
    
    if not agg_files:
        print(f"⚠ No aggregated features found for {patient_id}")
        return None
    
    agg_file = agg_files[0]
    
    try:
        with open(agg_file, 'rb') as f:
            data = pickle.load(f)
        print(f"✓ Loaded {patient_id}: {agg_file.name}")
        return data
    except Exception as e:
        print(f"✗ Error loading {patient_id}: {e}")
        return None

print("✓ Loading function defined")

✓ Loading function defined


## 9. Load All Available Patients

In [31]:
# Load all patients
all_patient_data = {}

print("Loading patient data...\n")
for patient_id in ALL_PATIENTS:
    data = load_patient_features(patient_id)
    if data is not None:
        all_patient_data[patient_id] = data

print(f"\n✓ Loaded {len(all_patient_data)} / {len(ALL_PATIENTS)} patients")
print(f"\nLoaded patients: {list(all_patient_data.keys())}")

Loading patient data...

✓ Loaded 0cGdk9: aggregated_features_holdL.pkl
✓ Loaded 2IU8mi: aggregated_features_holdL.pkl
⚠ Directory not found: AB2PeX
⚠ Directory not found: AbzsOg
⚠ Directory not found: FYbcap
⚠ Directory not found: PuPVlx
✓ Loaded QZTsn6: aggregated_features_holdL.pkl
⚠ No aggregated features found for dCsWjQ
⚠ Directory not found: gNX5yb
✓ Loaded i4oK0F: aggregated_features_holdL.pkl
✓ Loaded 2IhVOz: aggregated_features_holdR.pkl
⚠ No aggregated features found for BYJoWR
⚠ Directory not found: VopvKx
✓ Loaded jyC0j3: aggregated_features_holdR.pkl

✓ Loaded 6 / 14 patients

Loaded patients: ['0cGdk9', '2IU8mi', 'QZTsn6', 'i4oK0F', '2IhVOz', 'jyC0j3']


## 10. Create Comparison DataFrame - Dominant Hemisphere Hold Task

In [32]:
# Extract features for dominant hemisphere during hold task
comparison_data_dominant_hold = []

for patient_id, data in all_patient_data.items():
    hold_type = data.get('_metadata', {}).get('hold_type', 'unknown')
    
    # medOn features
    if 'medOn' in data and 'dominant_hold' in data['medOn']:
        medOn_features = data['medOn']['dominant_hold']
        row = {
            'patient_id': patient_id,
            'hold_type': hold_type,
            'medication_state': 'medOn',
            'hemisphere': 'dominant',
            'condition': 'hold',
            **medOn_features
        }
        comparison_data_dominant_hold.append(row)
    
    # medOff features
    if 'medOff' in data and 'dominant_hold' in data['medOff']:
        medOff_features = data['medOff']['dominant_hold']
        row = {
            'patient_id': patient_id,
            'hold_type': hold_type,
            'medication_state': 'medOff',
            'hemisphere': 'dominant',
            'condition': 'hold',
            **medOff_features
        }
        comparison_data_dominant_hold.append(row)

df_dominant_hold = pd.DataFrame(comparison_data_dominant_hold)

print(f"✓ Created comparison dataframe: {df_dominant_hold.shape}")
print(f"\nColumns (first 15): {df_dominant_hold.columns.tolist()[:15]}")

# Display first few rows with available columns
display_cols = ['patient_id', 'hold_type', 'medication_state', 'hemisphere', 'persistence_entropy_mean']
# Add h1 feature count if available
if 'h1_feature_count_mean' in df_dominant_hold.columns:
    display_cols.append('h1_feature_count_mean')

print(f"\nFirst few rows:")
display(df_dominant_hold[display_cols])

✓ Created comparison dataframe: (11, 250)

Columns (first 15): ['patient_id', 'hold_type', 'medication_state', 'hemisphere', 'condition', 'persistence_entropy_mean', 'persistence_entropy_std', 'persistence_entropy_min', 'persistence_entropy_max', 'persistence_entropy_median', 'persistence_entropy_range', 'persistence_entropy_q25', 'persistence_entropy_q75', 'persistence_entropy_iqr', 'h0_feature_count_mean']

First few rows:


,patient_id,hold_type,medication_state,hemisphere,persistence_entropy_mean,h1_feature_count_mean
0,0cGdk9,holdL,medOn,right,6.5607,218.2000
1,0cGdk9,holdL,medOff,right,6.4888,216.5000
2,2IU8mi,holdL,medOff,right,6.7329,214.6000
3,QZTsn6,holdL,medOn,right,6.4544,218.7500
4,QZTsn6,holdL,medOff,right,6.8357,236.4000
5,i4oK0F,holdL,medOn,right,6.4704,219.4000
6,i4oK0F,holdL,medOff,right,6.4688,203.4000
7,2IhVOz,holdR,medOn,left,6.3227,199.8000
8,2IhVOz,holdR,medOff,left,6.5043,210.0000
9,jyC0j3,holdR,medOn,left,6.2687,198.6667


## 11. Create Comparison DataFrame - Non-Dominant Hemisphere Hold Task

In [33]:
# Extract features for non-dominant hemisphere during hold task
comparison_data_nondominant_hold = []

for patient_id, data in all_patient_data.items():
    hold_type = data.get('_metadata', {}).get('hold_type', 'unknown')
    
    # medOn features
    if 'medOn' in data and 'nondominant_hold' in data['medOn']:
        medOn_features = data['medOn']['nondominant_hold']
        row = {
            'patient_id': patient_id,
            'hold_type': hold_type,
            'medication_state': 'medOn',
            'hemisphere': 'nondominant',
            'condition': 'hold',
            **medOn_features
        }
        comparison_data_nondominant_hold.append(row)
    
    # medOff features
    if 'medOff' in data and 'nondominant_hold' in data['medOff']:
        medOff_features = data['medOff']['nondominant_hold']
        row = {
            'patient_id': patient_id,
            'hold_type': hold_type,
            'medication_state': 'medOff',
            'hemisphere': 'nondominant',
            'condition': 'hold',
            **medOff_features
        }
        comparison_data_nondominant_hold.append(row)

df_nondominant_hold = pd.DataFrame(comparison_data_nondominant_hold)

print(f"✓ Created comparison dataframe: {df_nondominant_hold.shape}")

# Display first few rows with available columns
display_cols = ['patient_id', 'hold_type', 'medication_state', 'hemisphere', 'persistence_entropy_mean']
# Add h1 feature count if available
if 'h1_feature_count_mean' in df_nondominant_hold.columns:
    display_cols.append('h1_feature_count_mean')

print(f"\nFirst few rows:")
display(df_nondominant_hold[display_cols].head(10))

✓ Created comparison dataframe: (11, 250)

First few rows:


,patient_id,hold_type,medication_state,hemisphere,persistence_entropy_mean,h1_feature_count_mean
0,0cGdk9,holdL,medOn,left,6.4798,213.8000
1,0cGdk9,holdL,medOff,left,6.5132,222.7500
2,2IU8mi,holdL,medOff,left,6.5995,218.4000
3,QZTsn6,holdL,medOn,left,6.5178,213.7500
4,QZTsn6,holdL,medOff,left,6.5372,215.4000
5,i4oK0F,holdL,medOn,left,6.7909,228.2000
6,i4oK0F,holdL,medOff,left,6.4771,218.8000
7,2IhVOz,holdR,medOn,right,6.5550,212.6000
8,2IhVOz,holdR,medOff,right,6.5755,215.0000
9,jyC0j3,holdR,medOn,right,6.6023,222.3333


## 12. Create Combined DataFrame - All Conditions

In [34]:
# Combine dominant and non-dominant for comprehensive analysis
df_all_hold = pd.concat([df_dominant_hold, df_nondominant_hold], ignore_index=True)

print(f"✓ Combined dataframe: {df_all_hold.shape}")
print(f"\nData summary:")
print(f"  Unique patients: {df_all_hold['patient_id'].nunique()}")
print(f"  Medication states: {df_all_hold['medication_state'].unique()}")
print(f"  Hemispheres: {df_all_hold['hemisphere'].unique()}")
print(f"  Hold types: {df_all_hold['hold_type'].unique()}")

✓ Combined dataframe: (22, 250)

Data summary:
  Unique patients: 6
  Medication states: ['medOn' 'medOff']
  Hemispheres: ['right' 'left']
  Hold types: ['holdL' 'holdR']


## 13. Quick Statistical Comparison - Persistence Entropy

In [35]:
# Compare persistence entropy between medOn and medOff (dominant hemisphere)
if not df_dominant_hold.empty and 'persistence_entropy_mean' in df_dominant_hold.columns:
    medOn_entropy = df_dominant_hold[df_dominant_hold['medication_state'] == 'medOn']['persistence_entropy_mean'].dropna()
    medOff_entropy = df_dominant_hold[df_dominant_hold['medication_state'] == 'medOff']['persistence_entropy_mean'].dropna()
    
    print("Persistence Entropy - Dominant Hemisphere (Hold Task)")
    print("="*60)
    print(f"\nmedOn:  {medOn_entropy.mean():.4f} ± {medOn_entropy.std():.4f} (n={len(medOn_entropy)})")
    print(f"medOff: {medOff_entropy.mean():.4f} ± {medOff_entropy.std():.4f} (n={len(medOff_entropy)})")
    
    if len(medOn_entropy) == len(medOff_entropy) and len(medOn_entropy) > 0:
        # Paired t-test (same patients, different conditions)
        t_stat, p_val = ttest_rel(medOn_entropy, medOff_entropy)
        print(f"\nPaired t-test: t={t_stat:.4f}, p={p_val:.4f}")
        
        if p_val < 0.05:
            print("✓ Significant difference (p < 0.05)")
        else:
            print("✗ No significant difference (p >= 0.05)")
else:
    print("⚠ Insufficient data for statistical comparison")

Persistence Entropy - Dominant Hemisphere (Hold Task)

medOn:  6.4154 ± 0.1181 (n=5)
medOff: 6.5760 ± 0.1667 (n=6)


## 14. Summary Statistics Table

In [ ]:
# Create summary statistics for key features
if not df_all_hold.empty:
    # Comprehensive list of SCALAR features to compare
    # Note: persistence_landscape, betti_curve, and heat_kernel are arrays, so excluded here
    key_features = [
        'persistence_entropy_mean',
        'h0_feature_count_mean', 'h1_feature_count_mean', 'h2_feature_count_mean', 'h3_feature_count_mean',
        'h0_avg_lifespan_mean', 'h1_avg_lifespan_mean', 'h2_avg_lifespan_mean', 'h3_avg_lifespan_mean',
        'h0_max_lifespan_mean', 'h1_max_lifespan_mean', 'h2_max_lifespan_mean', 'h3_max_lifespan_mean',
        'h0_std_lifespan_mean', 'h1_std_lifespan_mean', 'h2_std_lifespan_mean', 'h3_std_lifespan_mean',
        'h1_avg_birth_mean', 'h1_avg_death_mean',
        'h2_avg_birth_mean', 'h2_avg_death_mean',
        'h3_avg_birth_mean', 'h3_avg_death_mean'
    ]
    
    # Filter to features that exist
    available_features = [f for f in key_features if f in df_all_hold.columns]
    
    if available_features:
        summary = df_all_hold.groupby(['medication_state', 'hemisphere'])[available_features].agg(['mean', 'std', 'count'])
        print("\nSummary Statistics by Medication State and Hemisphere")
        print("="*80)
        display(summary)
        
        print(f"\n✓ Showing statistics for {len(available_features)} scalar features")
        print("\nNote: Array-based features (persistence_landscape, betti_curve, heat_kernel)")
        print("      are available but require special handling (L2 norm, vectorization, etc.)")
    else:
        print("⚠ No key features available in dataframe")
        print(f"Available columns: {df_all_hold.columns.tolist()[:20]}")
else:
    print("⚠ No data available for summary")

## 15. Save Processed DataFrames

In [ ]:
# Save dataframes for further analysis
output_dir = BASE_DIR / 'analysis_output'
output_dir.mkdir(exist_ok=True)

# Save as CSV
if not df_all_hold.empty:
    df_all_hold.to_csv(output_dir / 'all_hold_features_comparison.csv', index=False)
    print(f"✓ Saved: {output_dir / 'all_hold_features_comparison.csv'}")

if not df_dominant_hold.empty:
    df_dominant_hold.to_csv(output_dir / 'dominant_hold_features_comparison.csv', index=False)
    print(f"✓ Saved: {output_dir / 'dominant_hold_features_comparison.csv'}")

if not df_nondominant_hold.empty:
    df_nondominant_hold.to_csv(output_dir / 'nondominant_hold_features_comparison.csv', index=False)
    print(f"✓ Saved: {output_dir / 'nondominant_hold_features_comparison.csv'}")

# Also save as pickle for preserving dtypes
with open(output_dir / 'all_patient_data.pkl', 'wb') as f:
    pickle.dump(all_patient_data, f)
print(f"✓ Saved: {output_dir / 'all_patient_data.pkl'}")

print("\n✓ All dataframes saved successfully")

---

# Part 2: Two-Patient Comparison Pipeline

This section allows flexible comparison between any two patients across different states and conditions.

## 16. Select Patients for Comparison

In [37]:
# Select two patients to compare
# Change these IDs to compare different patients
patient1 = 'i4oK0F'  # First patient
patient2 = 'QZTsn6'  # Second patient

# Verify patients are loaded
print(f"Available patients: {list(all_patient_data.keys())}\n")

if patient1 in all_patient_data:
    print(f"✓ {patient1} loaded")
    print(f"  Hold type: {all_patient_data[patient1].get('_metadata', {}).get('hold_type', 'unknown')}")
else:
    print(f"✗ {patient1} not available")

if patient2 in all_patient_data:
    print(f"✓ {patient2} loaded")
    print(f"  Hold type: {all_patient_data[patient2].get('_metadata', {}).get('hold_type', 'unknown')}")
else:
    print(f"✗ {patient2} not available")

Available patients: ['0cGdk9', '2IU8mi', 'QZTsn6', 'i4oK0F', '2IhVOz', 'jyC0j3']

✓ i4oK0F loaded
  Hold type: holdL
✓ QZTsn6 loaded
  Hold type: holdL


## 17. Helper Function for Two-Patient Comparison

In [38]:
def compare_two_patients(pid1, pid2, med_state='medOn', hemisphere='dominant', condition='hold', 
                          feature_list=None):
    """
    Compare features between two patients for a specific condition.
    
    Args:
        pid1, pid2: Patient IDs
        med_state: 'medOn' or 'medOff'
        hemisphere: 'dominant', 'nondominant', 'left', or 'right'
        condition: 'hold' or 'resting'
        feature_list: List of features to compare (None = all features)
    
    Returns:
        pandas.DataFrame: Comparison table
    """
    if pid1 not in all_patient_data or pid2 not in all_patient_data:
        print(f"Error: One or both patients not available")
        return None
    
    # Get the key for accessing features
    key = f"{hemisphere}_{condition}"
    
    # Extract features for both patients
    features1 = all_patient_data[pid1].get(med_state, {}).get(key, {})
    features2 = all_patient_data[pid2].get(med_state, {}).get(key, {})
    
    if not features1 or not features2:
        print(f"Error: Features not available for {med_state}/{hemisphere}/{condition}")
        return None
    
    # If no feature list provided, use all common features
    if feature_list is None:
        feature_list = [k for k in features1.keys() if k in features2 and k not in ['hemisphere', 'condition']]
    
    # Build comparison table
    comparison = []
    for feature in feature_list:
        val1 = features1.get(feature)
        val2 = features2.get(feature)
        
        # Compute difference if both values are numeric
        diff = None
        if val1 is not None and val2 is not None and isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
            diff = val2 - val1
        
        comparison.append({
            'feature': feature,
            f'{pid1}': val1,
            f'{pid2}': val2,
            'difference': diff
        })
    
    df_comp = pd.DataFrame(comparison)
    return df_comp

print("✓ Comparison function defined")

✓ Comparison function defined


## 18. Compare Dominant Hemisphere - medOn Hold Task

In [ ]:
# Compare dominant hemisphere features during hold task (medOn)
print(f"Comparing {patient1} vs {patient2} - Dominant Hemisphere, medOn, Hold Task")
print("="*80)

# Select key SCALAR features for comparison (array features excluded)
key_features = [
    'persistence_entropy_mean',
    'h0_feature_count_mean', 'h1_feature_count_mean', 'h2_feature_count_mean', 'h3_feature_count_mean',
    'h0_avg_lifespan_mean', 'h1_avg_lifespan_mean', 'h2_avg_lifespan_mean', 'h3_avg_lifespan_mean',
    'h0_max_lifespan_mean', 'h1_max_lifespan_mean', 'h2_max_lifespan_mean', 'h3_max_lifespan_mean',
    'h0_std_lifespan_mean', 'h1_std_lifespan_mean', 'h2_std_lifespan_mean', 'h3_std_lifespan_mean',
    'h1_avg_birth_mean', 'h1_avg_death_mean',
    'h2_avg_birth_mean', 'h2_avg_death_mean',
    'h3_avg_birth_mean', 'h3_avg_death_mean'
]

comp_dominant_medOn = compare_two_patients(
    patient1, patient2, 
    med_state='medOn', 
    hemisphere='dominant', 
    condition='hold',
    feature_list=key_features
)

if comp_dominant_medOn is not None:
    display(comp_dominant_medOn)
    
    # Highlight largest differences
    if 'difference' in comp_dominant_medOn.columns:
        comp_dominant_medOn['abs_diff'] = comp_dominant_medOn['difference'].abs()
        top_diff = comp_dominant_medOn.nlargest(5, 'abs_diff')
        print(f"\n✓ Top 5 largest differences:")
        display(top_diff[['feature', f'{patient1}', f'{patient2}', 'difference']])
        
    # Note about array features
    print("\nNote: Array-based features (persistence_landscape, betti_curve, heat_kernel)")
    print("      require special handling and are shown separately in other cells.")

## 19. Compare Non-Dominant Hemisphere - medOn Hold Task

In [ ]:
# Compare non-dominant hemisphere features during holding task (medOn)
print(f"Comparing {patient1} vs {patient2} - Non-Dominant Hemisphere, medOn, Holding")
print("="*80)

comp_nondominant_medOn = compare_two_patients(
    patient1, patient2, 
    med_state='medOn', 
    hemisphere='nondominant', 
    condition='hold',
    feature_list=key_features  # Uses key_features defined in previous cell
)

if comp_nondominant_medOn is not None:
    display(comp_nondominant_medOn)
    
    # Highlight largest differences (changed from top 3 to top 5 for consistency)
    if 'difference' in comp_nondominant_medOn.columns:
        comp_nondominant_medOn['abs_diff'] = comp_nondominant_medOn['difference'].abs()
        top_diff = comp_nondominant_medOn.nlargest(5, 'abs_diff')
        print(f"\n✓ Top 5 largest differences:")
        display(top_diff[['feature', f'{patient1}', f'{patient2}', 'difference']])

## 20. Compare Dominant Hemisphere - medOff Hold Task

In [ ]:
# Compare dominant hemisphere features during hold task (medOff)
print(f"Comparing {patient1} vs {patient2} - Dominant Hemisphere, medOff, hold")
print("="*80)

comp_dominant_medOff = compare_two_patients(
    patient1, patient2, 
    med_state='medOff', 
    hemisphere='dominant', 
    condition='hold',
    feature_list=key_features  # Uses key_features defined in previous cell
)

if comp_dominant_medOff is not None:
    display(comp_dominant_medOff)
    
    # Highlight largest differences (changed from top 3 to top 5 for consistency)
    if 'difference' in comp_dominant_medOff.columns:
        comp_dominant_medOff['abs_diff'] = comp_dominant_medOff['difference'].abs()
        top_diff = comp_dominant_medOff.nlargest(5, 'abs_diff')
        print(f"\n✓ Top 5 largest differences:")
        display(top_diff[['feature', f'{patient1}', f'{patient2}', 'difference']])

## 21. Visualize Two-Patient Comparison

In [ ]:
# Visualize comparison for dominant hemisphere, medOn
if comp_dominant_medOn is not None and not comp_dominant_medOn.empty:
    # Select SCALAR features for visualization (array features excluded)
    viz_features = [
        'persistence_entropy_mean', 
        'h1_feature_count_mean', 
        'h2_feature_count_mean', 
        'h1_avg_lifespan_mean',
        'h2_avg_lifespan_mean',
        'h1_max_lifespan_mean',
        'h1_avg_birth_mean',
        'h1_avg_death_mean'
    ]
    
    df_viz = comp_dominant_medOn[comp_dominant_medOn['feature'].isin(viz_features)].copy()
    
    if not df_viz.empty:
        # Create subplots for each feature
        n_features = len(df_viz)
        n_rows = (n_features + 2) // 3  # Calculate rows needed for 3 columns
        n_cols = min(3, n_features)
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1) if n_features > 1 else np.array([[axes]])
        axes = axes.flatten()
        
        for idx, (_, row) in enumerate(df_viz.iterrows()):
            if idx < len(axes):
                ax = axes[idx]
                feature_name = row['feature']
                values = [row[patient1], row[patient2]]
                colors = ['#3498db', '#e74c3c']
                
                ax.bar([patient1, patient2], values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
                ax.set_title(feature_name.replace('_', ' ').title(), fontsize=10, fontweight='bold')
                ax.set_ylabel('Value', fontsize=9)
                ax.tick_params(axis='x', rotation=45, labelsize=8)
                ax.grid(axis='y', alpha=0.3)
                
                # Add value labels on bars
                for i, (patient, val) in enumerate(zip([patient1, patient2], values)):
                    if val is not None and not pd.isna(val):
                        ax.text(i, val, f'{val:.3f}', ha='center', va='bottom', fontsize=8)
        
        # Hide unused subplots
        for idx in range(n_features, len(axes)):
            axes[idx].set_visible(False)
        
        plt.suptitle(f'Feature Comparison: {patient1} vs {patient2}\\nDominant Hemisphere, medOn, Hold Task (Scalar Features)',
                     fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("\nNote: Array-based features (persistence_landscape, betti_curve, heat_kernel)")
        print("      require advanced visualization techniques (heatmaps, line plots, etc.)")
    else:
        print("No features available for visualization")

## 22. Single Patient: medOn vs medOff Comparison (Both Hemispheres)

In [ ]:
# Compare medOn vs medOff for a single patient across both hemispheres
selected_patient = patient1  # Change to patient1 or patient2

if selected_patient in all_patient_data:
    print(f"Patient: {selected_patient}")
    print(f"Hold type: {all_patient_data[selected_patient].get('_metadata', {}).get('hold_type', 'unknown')}")
    print("="*80)
    
    # Create comparison dataframe with scalar features only
    comparison_data = []
    
    for hemisphere in ['dominant', 'nondominant']:
        for med_state in ['medOn', 'medOff']:
            features = all_patient_data[selected_patient].get(med_state, {}).get(f'{hemisphere}_hold', {})
            
            if features:
                row = {
                    'hemisphere': hemisphere,
                    'med_state': med_state,
                    'persistence_entropy': features.get('persistence_entropy_mean'),
                    'h1_count': features.get('h1_feature_count_mean'),
                    'h2_count': features.get('h2_feature_count_mean'),
                    'h1_avg_lifespan': features.get('h1_avg_lifespan_mean'),
                    'h2_avg_lifespan': features.get('h2_avg_lifespan_mean'),
                    'h1_max_lifespan': features.get('h1_max_lifespan_mean'),
                    'h2_max_lifespan': features.get('h2_max_lifespan_mean'),
                    'h1_avg_birth': features.get('h1_avg_birth_mean'),
                    'h1_avg_death': features.get('h1_avg_death_mean')
                }
                comparison_data.append(row)
    
    df_single_patient = pd.DataFrame(comparison_data)
    
    if not df_single_patient.empty:
        print(f"\n{selected_patient} - Feature Comparison Table:")
        display(df_single_patient)
        
        # Calculate differences (medOn - medOff)
        print(f"\nDifferences (medOn - medOff):")
        for hemisphere in ['dominant', 'nondominant']:
            print(f"\n{hemisphere.upper()} Hemisphere:")
            medOn_data = df_single_patient[(df_single_patient['hemisphere'] == hemisphere) & 
                                           (df_single_patient['med_state'] == 'medOn')]
            medOff_data = df_single_patient[(df_single_patient['hemisphere'] == hemisphere) & 
                                            (df_single_patient['med_state'] == 'medOff')]
            
            if not medOn_data.empty and not medOff_data.empty:
                for col in ['persistence_entropy', 'h1_count', 'h2_count', 'h1_avg_lifespan', 
                           'h2_avg_lifespan', 'h1_max_lifespan', 'h1_avg_birth', 'h1_avg_death']:
                    medOn_val = medOn_data[col].values[0]
                    medOff_val = medOff_data[col].values[0]
                    if medOn_val is not None and medOff_val is not None:
                        diff = medOn_val - medOff_val
                        print(f"  {col}: {safe_format(diff, '.4f')}")
        
        # Show array-based features separately using L2 norms
        print(f"\n\nArray-based Features (L2 Norms):")
        print("-" * 60)
        for hemisphere in ['dominant', 'nondominant']:
            print(f"\n{hemisphere.upper()} Hemisphere:")
            medOn_features = all_patient_data[selected_patient].get('medOn', {}).get(f'{hemisphere}_hold', {})
            medOff_features = all_patient_data[selected_patient].get('medOff', {}).get(f'{hemisphere}_hold', {})
            
            # Persistence Landscape
            medOn_pl = medOn_features.get('persistence_landscape_mean')
            medOff_pl = medOff_features.get('persistence_landscape_mean')
            if medOn_pl is not None and medOff_pl is not None:
                try:
                    medOn_norm = np.linalg.norm(medOn_pl)
                    medOff_norm = np.linalg.norm(medOff_pl)
                    print(f"  Persistence Landscape: medOn={medOn_norm:.4f}, medOff={medOff_norm:.4f}, diff={medOn_norm-medOff_norm:.4f}")
                except:
                    pass
            
            # Betti Curve
            medOn_bc = medOn_features.get('betti_curve_mean')
            medOff_bc = medOff_features.get('betti_curve_mean')
            if medOn_bc is not None and medOff_bc is not None:
                try:
                    medOn_norm = np.linalg.norm(medOn_bc)
                    medOff_norm = np.linalg.norm(medOff_bc)
                    print(f"  Betti Curve: medOn={medOn_norm:.4f}, medOff={medOff_norm:.4f}, diff={medOn_norm-medOff_norm:.4f}")
                except:
                    pass
            
            # Heat Kernel
            medOn_hk = medOn_features.get('heat_kernel_mean')
            medOff_hk = medOff_features.get('heat_kernel_mean')
            if medOn_hk is not None and medOff_hk is not None:
                try:
                    medOn_norm = np.linalg.norm(medOn_hk)
                    medOff_norm = np.linalg.norm(medOff_hk)
                    print(f"  Heat Kernel: medOn={medOn_norm:.4f}, medOff={medOff_norm:.4f}, diff={medOn_norm-medOff_norm:.4f}")
                except:
                    pass
    else:
        print(f"No data available for {selected_patient}")
else:
    print(f"Patient {selected_patient} not loaded")

## 23. Visualize Single Patient: medOn vs medOff

In [ ]:
# Visualize medOn vs medOff for single patient (scalar features only)
if not df_single_patient.empty:
    # Select scalar features for visualization
    # Array-based features (persistence_landscape, betti_curve, heat_kernel) excluded
    features_to_plot = [
        'persistence_entropy', 
        'h1_count', 
        'h2_count', 
        'h1_avg_lifespan',
        'h2_avg_lifespan',
        'h1_max_lifespan',
        'h2_max_lifespan',
        'h1_avg_birth',
        'h1_avg_death'
    ]
    
    # Filter to features that have data
    available_features = [f for f in features_to_plot if f in df_single_patient.columns 
                          and df_single_patient[f].notna().any()]
    
    if available_features:
        # Create grid layout for features
        n_features = len(available_features)
        n_rows = (n_features + 2) // 3  # 3 columns
        n_cols = min(3, n_features)
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1 and n_cols == 1:
            axes = np.array([[axes]])
        elif n_rows == 1:
            axes = axes.reshape(1, -1)
        elif n_cols == 1:
            axes = axes.reshape(-1, 1)
        axes = axes.flatten()
        
        for idx, feature in enumerate(available_features):
            if idx < len(axes):
                ax = axes[idx]
                
                # Prepare data for grouped bar chart
                dominant_medOn = df_single_patient[(df_single_patient['hemisphere'] == 'dominant') & 
                                                   (df_single_patient['med_state'] == 'medOn')][feature].values
                dominant_medOff = df_single_patient[(df_single_patient['hemisphere'] == 'dominant') & 
                                                    (df_single_patient['med_state'] == 'medOff')][feature].values
                nondominant_medOn = df_single_patient[(df_single_patient['hemisphere'] == 'nondominant') & 
                                                      (df_single_patient['med_state'] == 'medOn')][feature].values
                nondominant_medOff = df_single_patient[(df_single_patient['hemisphere'] == 'nondominant') & 
                                                       (df_single_patient['med_state'] == 'medOff')][feature].values
                
                x = np.arange(2)  # Dominant, Nondominant
                width = 0.35
                
                # Plot bars
                if len(dominant_medOn) > 0 and len(dominant_medOff) > 0:
                    bars1 = ax.bar(x - width/2, 
                                   [dominant_medOn[0], nondominant_medOn[0] if len(nondominant_medOn) > 0 else 0], 
                                   width, label='medOn', color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=1.2)
                    bars2 = ax.bar(x + width/2, 
                                   [dominant_medOff[0], nondominant_medOff[0] if len(nondominant_medOff) > 0 else 0], 
                                   width, label='medOff', color='#e74c3c', alpha=0.8, edgecolor='black', linewidth=1.2)
                    
                    # Add value labels on bars
                    for bars in [bars1, bars2]:
                        for bar in bars:
                            height = bar.get_height()
                            if not pd.isna(height) and height != 0:
                                ax.text(bar.get_x() + bar.get_width()/2., height,
                                       f'{height:.3f}',
                                       ha='center', va='bottom', fontsize=7)
                
                ax.set_xlabel('Hemisphere', fontsize=9)
                ax.set_ylabel('Value', fontsize=9)
                ax.set_title(feature.replace('_', ' ').title(), fontweight='bold', fontsize=10)
                ax.set_xticks(x)
                ax.set_xticklabels(['Dominant', 'Non-Dominant'])
                ax.legend(fontsize=8)
                ax.grid(axis='y', alpha=0.3)
        
        # Hide unused subplots
        for idx in range(len(available_features), len(axes)):
            axes[idx].set_visible(False)
        
        plt.suptitle(f'{selected_patient} - medOn vs medOff Comparison (Hold Task, Scalar Features)', 
                     fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("\nNote: Array-based features (persistence_landscape, betti_curve, heat_kernel)")
        print("      are shown separately as L2 norms in the comparison table above.")
    else:
        print("No features available for visualization")
else:
    print("No data available for visualization")

## 24. Custom Flexible Comparison

Use this cell to create custom comparisons with different parameters.

In [43]:
# Custom comparison - modify these parameters as needed
custom_patient1 = 'i4oK0F'
custom_patient2 = '0cGdk9'
custom_med_state = 'medOn'      # 'medOn' or 'medOff'
custom_hemisphere = 'dominant'   # 'dominant', 'nondominant', 'left', or 'right'
custom_condition = 'hold'        # 'hold' or 'resting'

# Custom feature list (optional - set to None for all features)
custom_features = [
    'persistence_entropy_mean',
    'h0_feature_count_mean',
    'h1_feature_count_mean',
    'h2_feature_count_mean',
    'h3_feature_count_mean',
    'h1_avg_lifespan_mean',
    'h1_max_lifespan_mean'
]

print(f"Custom Comparison: {custom_patient1} vs {custom_patient2}")
print(f"Parameters: {custom_med_state} | {custom_hemisphere} hemisphere | {custom_condition} task")
print("="*80)

custom_comp = compare_two_patients(
    custom_patient1, 
    custom_patient2,
    med_state=custom_med_state,
    hemisphere=custom_hemisphere,
    condition=custom_condition,
    feature_list=custom_features
)

if custom_comp is not None:
    display(custom_comp)
else:
    print("Comparison failed - check that both patients have data for the specified parameters")

Custom Comparison: i4oK0F vs 0cGdk9
Parameters: medOn | dominant hemisphere | hold task


,feature,i4oK0F,0cGdk9,difference
0,persistence_entropy_mean,6.4704,6.5607,0.0903
1,h0_feature_count_mean,249.0000,249.0000,0.0000
2,h1_feature_count_mean,219.4000,218.2000,-1.2000
3,h2_feature_count_mean,106.0000,119.0000,13.0000
4,h3_feature_count_mean,34.0000,36.6000,2.6000
5,h1_avg_lifespan_mean,0.2259,0.1358,-0.0901
6,h1_max_lifespan_mean,0.8108,0.4654,-0.3454


## 25. Summary: Available Comparisons

This notebook provides multiple comparison pipelines:

### 1. **Multi-Patient Analysis** (Sections 1-15)
- Loads all available patient data
- Creates comprehensive DataFrames for all patients
- Compares features across the entire cohort
- Exports aggregated comparison tables

### 2. **Two-Patient Direct Comparison** (Sections 16-21)
- Compare any two patients side-by-side
- Includes both dominant and non-dominant hemispheres
- Covers medOn and medOff states
- Highlights largest feature differences

### 3. **Single Patient Medication State Comparison** (Sections 22-23)
- Compare medOn vs medOff within a single patient
- Includes both hemispheres
- Visualizes differences between medication states

### 4. **Custom Flexible Comparison** (Section 24)
- Fully customizable parameters:
  - Any two patients
  - Any medication state (medOn/medOff)
  - Any hemisphere (dominant/nondominant/left/right)
  - Any condition (hold/resting)
  - Custom feature selection

### Key Variables to Modify:
- `patient1`, `patient2`: Set patient IDs for comparison
- `selected_patient`: Choose patient for medOn vs medOff analysis
- `custom_patient1`, `custom_patient2`: Custom comparison patient IDs
- `custom_med_state`, `custom_hemisphere`, `custom_condition`: Custom comparison parameters

### Available Features:
- **Persistence Entropy**: Overall topological complexity
- **H0, H1, H2, H3 Feature Counts**: Number of topological features per dimension
- **Average Lifespans**: Mean persistence of features
- **Max Lifespans**: Longest-living features
- **Standard Deviations**: Variability measures for all features

### Next Steps:
1. Run statistical tests (t-tests, Wilcoxon) on feature differences
2. Apply dimensionality reduction (PCA) for multi-feature analysis
3. Create correlation matrices between features
4. Implement distance-based comparisons using Wasserstein/Bottleneck metrics
5. Apply analysis pathways from ANALYSIS_METHODOLOGY.md